In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
object_names = []
with open("./coco.names",'r') as f:
    object_names = f.read().splitlines()

In [3]:
yolo = cv2.dnn.readNet("./model/yolov3.weights","./model/yolov3.cfg")

In [6]:
vid = cv2.VideoCapture(0)
# vid.set(cv2.CAP_PROP_BUFFERSIZE, 2)
font = cv2.FONT_HERSHEY_PLAIN
colours = np.random.uniform(0, 255, (len(object_names), 3))

while(True):
    ret, img = vid.read()
    # img = cv2.imread("./Images/2.jpeg")
    height, width, _ = img.shape
    new_height = 320
    new_width = 320
    new_img = cv2.dnn.blobFromImage(img, 1/255, (new_height, new_width), (0, 0, 0), swapRB=True, crop=False)
    yolo.setInput(new_img)
    output_object_names = yolo.getUnconnectedOutLayersNames()
    output_layer = yolo.forward(output_object_names)
    output_object_names = yolo.getUnconnectedOutLayersNames()
    output_layer = yolo.forward(output_object_names)
    
    layers = []
    confidence_levels = []
    object_ids = []
    for layer in output_layer:
        for detection in layer:
            score = detection[5:]
            object_id = np.argmax(score)
            confidence_level = score[object_id]
            if confidence_level > 0.7:
                w = int(detection[0]*width)
                h = int(detection[1]*height)
                x = int(int(detection[0]*width) - w/2)
                y = int(int(detection[1]*height) - h/2)
                layers.append([x, y, w, h])
                confidence_levels.append(float(confidence_level))
                object_ids.append(object_id)
    indexes = cv2.dnn.NMSBoxes(layers, confidence_levels, 0.5, 0.4)
    if(len(indexes)!=0):
        for i in indexes.flatten():
            x,y,w,h = layers[i]
            label = str(object_names[object_ids[i]])
            confidence = str(round(confidence_levels[i],2))
            colour= colours[i]
            cv2.rectangle(img, (x, y), (x+w-10, y+h-20), colour, 2)
            cv2.putText(img,"{}-{}".format(label,confidence) ,(x,y+20),font,2,colour,2)
    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()
